## calculate the pO2 for each grid point at each depth for FOSI hindcast

In [1]:
import metabolic as mi
import xarray as xr
import numpy as np
import netCDF4
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
USER = os.environ['USER']
var1 = 'O2' # mmol/m3
var2 = 'TEMP' # deg C
var3 = 'SALT' # PSS
var = [var1,var2,var3]
a1,a2,b1,b2 = [240,380,125,320]

### DPLE hindcast

In [3]:
%%time
datadir = '/glade/campaign/cesm/collections/CESM1-DPLE/DPLE-FOSI/rerun_with_extension/timeseries'
dso = xr.Dataset()
#
for vv in var:
    filename = '%s/g.DPLE.GECOIAF.T62_g16.009.chey.pop.h.%s.024901-031812.nc' % (datadir,vv)
    ds_hind = xr.open_dataset(filename)
    # first 35 layers
    dso[vv] = ds_hind[vv].isel(z_t=range(0, 35), nlat=range(a1, a2), nlon=range(b1, b2)).compute()

CPU times: user 2min 1s, sys: 6.6 s, total: 2min 7s
Wall time: 2min 43s


In [4]:
#for vv in ['TLONG', 'TLAT', 'z_t', 'time', 'time_bound', 'KMT', 'TAREA', 'dz']:
#    dso[vv] = ds_hind[vv] 

### save T, S, O2

In [5]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dso.load()
dso.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_T_S_O2_1948-2017_monthly.nc', mode='w')

### calculate pO2 based on O2, T, S, depth

In [6]:
depth = ds_hind['z_t'].isel(z_t=range(0, 35))*dso[var1]/dso[var1]/1e2 # m

In [7]:
dso['depth'] = depth.transpose("time","z_t","nlat","nlon")

In [8]:
%%time
pO2 = mi.compute_pO2(dso[var1],dso[var2],dso[var3],dso['depth'])

CPU times: user 1min 26s, sys: 38.1 s, total: 2min 4s
Wall time: 2min 10s


### save pO2

In [9]:
dsf = xr.Dataset()
dsf['pO2'] = pO2

In [10]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dsf.load()

<xarray.Dataset>
Dimensions:  (z_t: 35, nlat: 140, nlon: 195, time: 840)
Coordinates:
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.277e+04 5.794e+04
    TLAT     (nlat, nlon) float64 14.96 14.96 14.97 14.97 ... 70.43 70.42 70.42
    TLONG    (nlat, nlon) float64 101.2 102.3 103.4 104.5 ... 318.6 319.2 319.7
    ULAT     (nlat, nlon) float64 15.13 15.13 15.14 15.14 ... 70.64 70.63 70.63
    ULONG    (nlat, nlon) float64 101.7 102.9 104.0 105.1 ... 318.9 319.5 320.0
  * time     (time) object 0249-02-01 00:00:00 ... 0319-01-01 00:00:00
Dimensions without coordinates: nlat, nlon
Data variables:
    pO2      (time, z_t, nlat, nlon) float32 nan nan nan nan ... nan nan nan nan

In [11]:
%%time
dsf.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_pO2_1948-2017_monthly.nc', mode='w')

CPU times: user 61 ms, sys: 606 ms, total: 667 ms
Wall time: 767 ms


## calculate yearly T,S,O2, and pO2

In [12]:
def xr_reshape(A, dim, newdims, coords):
    """ Reshape DataArray A to convert its dimension dim into sub-dimensions given by
    newdims and the corresponding coords.
    Example: Ar = xr_reshape(A, 'time', ['year', 'month'], [(2017, 2018), np.arange(12)]) """

    # Create a pandas MultiIndex from these labels
    ind = pd.MultiIndex.from_product(coords, names=newdims)

    # Replace the time index in the DataArray by this new index,
    A1 = A.copy()

    A1.coords[dim] = ind

    # Convert multiindex to individual dims using DataArray.unstack().
    # This changes dimension order! The new dimensions are at the end.
    A1 = A1.unstack(dim)

    # Permute to restore dimensions
    i = A.dims.index(dim)
    dims = list(A1.dims)

    for d in newdims[::-1]:
        dims.insert(i, d)

    for d in newdims:
        _ = dims.pop(-1)


    return A1.transpose(*dims)

### Calculate yearly T,S,O2

In [13]:
dsg = xr.Dataset()
for varr in [var1,var2,var3]:
    dsg[varr] = xr_reshape(dso[varr], 'time', ['year', 'mon'], [np.arange(70), np.arange(12)]).mean(dim='mon')

### Save yearly T,S,O2

In [14]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dsg.load()
dsg.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_T_S_O2_1948-2017_yearly.nc', mode='w')

### Calculate yearly pO2

In [15]:
dsh = xr.Dataset()
dsh['pO2'] = xr_reshape(dsf['pO2'], 'time', ['year', 'mon'], [np.arange(70), np.arange(12)]).mean(dim='mon')

### Save yearly pO2

In [16]:
dsh.load()
dsh.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_pO2_1948-2017_yearly.nc', mode='w')